# Download data

To run this notebook you will need to download the GW data.

`python download_GW170817_data.py`

# Demo

Use the L1 GW170817 data to demonstrate (i) glitch removal, (ii) bandpass filtering, and (iii) whitening in the wavelet time-frequency domain.

In [1]:
import numpy as np

import matplotlib.pyplot as plt

import WDM

# Load data

In [14]:
path = "../data/L-L1_GWOSC_4KHZ_R1-1187006835-4096.txt"

with open(path, 'r', encoding='utf-8') as f:
    for line in f:
        if line.startswith('#'):
            print(line.rstrip('\n'))
        else:
            break

data = np.loadtxt(path)

f_samp = 4096.0

gps_time = 1187006835.0 + np.arange(data.shape[0]) / f_samp

# Gravitational wave strain for GW170817_R1 for L1 (see http://losc.ligo.org)
# This file has 4096 samples per second
# starting GPS 1187006835 duration 4096


# Noise Estimatation

In [ ]:
def overlapping_windows(x, num_perseg, num_overlap):
    """
    Split input `x` into overlapping windows. Trailing samples that don't fill
    a full window are discarded.

    Parameters
    ----------
    x : ndarray
        Input array to split into windows.
    num_perseg : int
        Length of each segment.
    num_overlap : int
        Number of overlapping samples between segments.

    Returns
    -------
    y : ndarray 
        Array of shape (n_windows, num_perseg).
    """
    x = np.asarray(x)
    if x.ndim != 1:
        raise ValueError("x must be 1D")
    if num_perseg <= 0:
        raise ValueError("num_perseg must be positive")
    if not (0 <= num_overlap < num_perseg):
        raise ValueError("num_overlap must satisfy 0 <= overlap < num_perseg")

    step = num_perseg - num_overlap
    if x.size < num_perseg:
        return np.empty((0, num_perseg), dtype=x.dtype)

    return np.lib.stride_tricks.sliding_window_view(x, num_perseg)[::step]

In [41]:
num_perseg = 32*int(f_samp)
num_overlap = num_perseg // 10

overlapping_times = overlapping_windows(gps_time, num_perseg, num_overlap)
overlapping_data = overlapping_windows(data, num_perseg, num_overlap)

trigger = 1187008882.43

idx = np.argmax(np.any(overlapping_times>trigger, axis=1))

print(f"Splitting data in {overlapping_times.shape[0]} segements with " \
      f"{num_perseg} samples per segment. " \
      f"The signal is in segment number {idx}.")

Splitting data in 142 segements with 131072 samples per segment. The signal is in segment number 70.


In [45]:
wdm = WDM.code.discrete_wavelet_transform.WDM.WDM_transform(dt=1./f_samp, Nf=256, N=num_perseg)

In [51]:
i=10
wdm.forward_transform_fft(overlapping_data[i])

Array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0., -0.,  0., ..., -0.,  0., -0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [-0., -0.,  0., ..., -0.,  0.,  0.],
       [ 0.,  0.,  0., ..., -0.,  0.,  0.],
       [ 0., -0., -0., ..., -0.,  0., -0.]], dtype=float64)

In [49]:
overlapping_data

array([[ 7.14542541e-20,  6.52920412e-20,  6.84001118e-20, ...,
         1.82066611e-19,  1.81332041e-19,  1.83418042e-19],
       [ 1.43235544e-19,  1.32145068e-19,  1.28687309e-19, ...,
        -5.71866915e-20, -6.84137304e-20, -6.47657097e-20],
       [-1.58975370e-19, -1.46702942e-19, -1.49324594e-19, ...,
        -1.64243209e-19, -1.58017998e-19, -1.59059632e-19],
       ...,
       [ 9.95504035e-20,  1.04410231e-19,  1.02323583e-19, ...,
        -2.17949965e-19, -2.21499579e-19, -2.24383952e-19],
       [-1.65828941e-19, -1.62077696e-19, -1.60427753e-19, ...,
         5.92753089e-21,  1.33514438e-20,  1.23126493e-20],
       [ 5.56083871e-20,  5.34565484e-20,  5.57811377e-20, ...,
        -1.95209445e-19, -2.01402402e-19, -1.94379460e-19]],
      shape=(142, 131072))